In [1]:
input_filename = "input.txt"

In [2]:
from collections import Counter
from enum import Enum

# Part 1

In [3]:
class HandType(Enum):
    FIVE_OF_A_KIND = 6
    FOUR_OF_A_KIND = 5
    FULL_HOUSE = 4
    THREE_OF_A_KIND = 3
    TWO_PAIR = 2
    ONE_PAIR = 1
    HIGH_CARD = 0

    def __lt__(self, other):
        return self.value < other.value

In [4]:
class CamelHand:
    ALL_LABELS = "23456789TJQKA"  # weakest to strongest
    
    def __init__(self, hand: str, bid: int = 0):
        assert len(hand) == 5
        for letter in hand:
            assert letter in self.ALL_LABELS
        self.hand = hand
        self.bid = bid
        
        self.hand_set = Counter(hand)
        
    @classmethod
    def from_raw(cls, line: str):
        hand, raw_bid = line.split()
        return cls(hand, int(raw_bid))
    
    def __eq__(self, other):
        return self.hand == other.hand
    
    def __lt__(self, other):
        if self.hand_type == other.hand_type:
            # compare the cards in the hands
            for label1, label2 in zip(self.hand, other.hand):
                strength1 = self.ALL_LABELS.index(label1)
                strength2 = self.ALL_LABELS.index(label2)
                if strength1 == strength2:
                    continue
                return strength1 < strength2
            
        return self.hand_type < other.hand_type
    
    @property
    def hand_type(self) -> int:
        """
        Returns hand type enum member.
        """
        values = sorted(self.hand_set.values())
        
        if values == [5]:
            return HandType.FIVE_OF_A_KIND
        
        elif values == [1, 4]:
            return HandType.FOUR_OF_A_KIND
        
        elif values == [2, 3]:
            return HandType.FULL_HOUSE
        
        elif values == [1, 1, 3]:
            return HandType.THREE_OF_A_KIND
        
        elif values == [1, 2, 2]:
            return HandType.TWO_PAIR
        
        elif values == [1, 1, 1, 2]:
            return HandType.ONE_PAIR
        
        elif values == [1, 1, 1, 1, 1]:
            return HandType.HIGH_CARD
        
        else:
            raise Exception(f"Unexpected hand: {self.hand}")

        

def test_hand_types():
    assert CamelHand("AAAAA").hand_type == HandType.FIVE_OF_A_KIND
    assert CamelHand("AA8AA").hand_type == HandType.FOUR_OF_A_KIND
    assert CamelHand("23332").hand_type == HandType.FULL_HOUSE
    assert CamelHand("TTT98").hand_type == HandType.THREE_OF_A_KIND
    assert CamelHand("23432").hand_type == HandType.TWO_PAIR
    assert CamelHand("A23A4").hand_type == HandType.ONE_PAIR
    assert CamelHand("23456").hand_type == HandType.HIGH_CARD
    print("test_hand_types passed!")


def test_comparison():
    assert CamelHand("AAAAA") == CamelHand("AAAAA")
    assert CamelHand("AAAAA") > CamelHand("AKAAA")
    assert CamelHand("33332") > CamelHand("2AAAA")
    assert CamelHand("77788") < CamelHand("77888")
    assert CamelHand("23456") < CamelHand("77888")
    print("test_comparison passed!")


    
test_hand_types()
test_comparison()

test_hand_types passed!
test_comparison passed!


In [5]:
with open(input_filename) as input_file:
    hands = sorted([CamelHand.from_raw(line) for line in input_file.readlines()])

winnings = 0
for rank, hand in enumerate(hands, start=1):
    winnings += (hand.bid * rank)

winnings

250370104

# Part 2

In [6]:
class CamelHandWithJokers(CamelHand):
    ALL_LABELS = "J23456789TQKA"  # weakest to strongest

    @property
    def hand_type(self) -> int:
        super_hand_type = super().hand_type
        
        num_jokers = self.hand_set["J"]
        
        # Cases where the hand type definitely doesn't change
        if num_jokers == 0 or num_jokers == 5:
            return super_hand_type
        
        # No need to check for FIVE_OF_A_KIND because it'd be all jokers.
        
        if super_hand_type == HandType.FOUR_OF_A_KIND:
            if num_jokers == 1 or num_jokers == 4:
                return HandType.FIVE_OF_A_KIND
            else:
                raise Exception(f"Unexpected four of a kind joker hand: {self.hand}")
            
        if super_hand_type == HandType.FULL_HOUSE:
            return HandType.FIVE_OF_A_KIND
        
        if super_hand_type == HandType.THREE_OF_A_KIND:
            if num_jokers == 3 or num_jokers == 1:
                return HandType.FOUR_OF_A_KIND
            else:
                raise Exception(f"Unexpected three of a kind joker hand: {self.hand}")
        
        if super_hand_type == HandType.TWO_PAIR:
            if num_jokers == 2:
                return HandType.FOUR_OF_A_KIND
            elif num_jokers == 1:
                return HandType.FULL_HOUSE
            else:
                raise Exception(f"Unexpected two pair joker hand: {self.hand}")
                
        if super_hand_type == HandType.ONE_PAIR:
            if num_jokers == 2:
                return HandType.THREE_OF_A_KIND
            elif num_jokers == 1:
                return HandType.THREE_OF_A_KIND
            else:
                raise Exception(f"Unexpected one pair joker hand: {self.hand}")
                
        if super_hand_type == HandType.HIGH_CARD:
            if num_jokers == 1:
                return HandType.ONE_PAIR
            else:
                raise Exception(f"Unexpected high card joker hand: {self.hand}")
        
        raise Exception(f"Unexpected joker hand at end: {self.hand}")


def test_joker_hand_types():
    assert CamelHandWithJokers("JJJJJ").hand_type == HandType.FIVE_OF_A_KIND
    assert CamelHandWithJokers("AAAAA").hand_type == HandType.FIVE_OF_A_KIND
    assert CamelHandWithJokers("32T3K").hand_type == HandType.ONE_PAIR
    assert CamelHandWithJokers("KK677").hand_type == HandType.TWO_PAIR

    assert CamelHandWithJokers("QJJQ2").hand_type == HandType.FOUR_OF_A_KIND
    assert CamelHandWithJokers("KTJJT").hand_type == HandType.FOUR_OF_A_KIND
    assert CamelHandWithJokers("T55J5").hand_type == HandType.FOUR_OF_A_KIND
    assert CamelHandWithJokers("QQQJA").hand_type == HandType.FOUR_OF_A_KIND

    print("test_joker_hand_types passed!")


test_joker_hand_types()

test_joker_hand_types passed!


In [7]:
with open(input_filename) as input_file:
    hands = sorted([CamelHandWithJokers.from_raw(line) for line in input_file.readlines()])

winnings = 0
for rank, hand in enumerate(hands, start=1):
    winnings += (hand.bid * rank)

winnings

251735672